In [ ]:
# Importing libraries for file handling, data processing, image manipulation, and visualization
import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# PyTorch imports for building neural networks and data loading
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

# Progress bar for loops (e.g., training epochs)
from tqdm import tqdm


In [ ]:
# Define dataset paths and hyperparameters for model training

DATA_DIR = '/kaggle/input/soil-classification-part-2/soil_competition-2025/'  # Base directory for dataset

# Paths to CSV files containing labels and IDs for train and test sets
TRAIN_CSV = os.path.join(DATA_DIR, 'train_labels.csv')
TRAIN_IMG_DIR = os.path.join(DATA_DIR, 'train')
TEST_CSV = os.path.join(DATA_DIR, 'test_ids.csv')
TEST_IMG_DIR = os.path.join(DATA_DIR, 'test')

# Image size to resize all input images (128x128 pixels)
IMG_SIZE = 128

# Training parameters
BATCH_SIZE = 64     # Number of samples per batch during training
EPOCHS = 20         # Number of full passes through the training dataset
LR = 1e-3           # Learning rate for optimizer

# Select device: GPU if available, otherwise CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
# Custom PyTorch Dataset class to load soil images for training/testing

class SoilImageDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        """
        Args:
            dataframe (pd.DataFrame): DataFrame containing image metadata (e.g., image IDs).
            img_dir (str): Directory where images are stored.
            transform (callable, optional): Optional image transformations (augmentation, normalization).
        """
        self.df = dataframe.reset_index(drop=True)  # Reset index to ensure sequential indexing
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        # Return total number of samples in the dataset
        return len(self.df)

    def __getitem__(self, idx):
        # Fetch image ID for given index
        img_id = self.df.loc[idx, 'image_id']

        # Construct full image path and load the image
        img_path = os.path.join(self.img_dir, img_id)
        image = Image.open(img_path).convert('RGB')  # Ensure image is in RGB format

        # Apply transformations if any are specified
        if self.transform:
            image = self.transform(image)

        # Return processed image tensor
        return image


In [ ]:
# Define basic image transformations: resize images and convert them to PyTorch tensors
transforms_basic = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),  # Resize all images to fixed size (128x128)
    transforms.ToTensor(),                     # Convert PIL Image to PyTorch tensor (C x H x W), values scaled [0,1]
])

# Load training data CSV with labels
train_df = pd.read_csv(TRAIN_CSV)

# Initialize custom dataset for training images with defined transformations
train_dataset = SoilImageDataset(train_df, TRAIN_IMG_DIR, transform=transforms_basic)

# Create DataLoader to batch training data and shuffle it for randomness
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Load test data CSV (which only contains image IDs, no labels)
test_df = pd.read_csv(TEST_CSV)

# Initialize dataset and DataLoader for test images (batch size 1, no shuffling)
test_dataset = SoilImageDataset(test_df, TEST_IMG_DIR, transform=transforms_basic)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


In [ ]:
# Define a simple convolutional Autoencoder architecture using PyTorch

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        # Encoder: progressively downsamples the input image and extracts features
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=2, padding=1),  # Input channels=3 (RGB), output=16 feature maps
                                                      # Output size: [16, 64, 64] (128/2=64)
            nn.ReLU(),                                 # Activation function
            
            nn.Conv2d(16, 32, 3, stride=2, padding=1), # Output: [32, 32, 32]
            nn.ReLU(),
            
            nn.Conv2d(32, 64, 3, stride=2, padding=1), # Output: [64, 16, 16]
            nn.ReLU()
        )
        
        # Decoder: upsamples the encoded features back to original image size
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1), # Upsample to [32, 32, 32]
            nn.ReLU(),
            
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1), # Upsample to [16, 64, 64]
            nn.ReLU(),
            
            nn.ConvTranspose2d(16, 3, 3, stride=2, padding=1, output_padding=1),  # Upsample to [3, 128, 128]
            nn.Sigmoid()  # Output values scaled between 0 and 1 for image reconstruction
        )

    def forward(self, x):
        # Forward pass: encode input then decode to reconstruct image
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [ ]:
# Initialize the Autoencoder model and move it to the selected device (GPU or CPU)
model = Autoencoder().to(device)

# Define the loss function as Mean Squared Error (MSE), suitable for reconstruction tasks
criterion = nn.MSELoss()

# Use Adam optimizer for training, with the specified learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# Training loop over defined number of epochs
for epoch in range(EPOCHS):
    model.train()  # Set model to training mode
    epoch_loss = 0  # Initialize variable to accumulate loss over the epoch

    # Iterate over batches of images from the training DataLoader
    for images in train_loader:
        images = images.to(device)            # Move batch to GPU/CPU
        outputs = model(images)                # Forward pass: get reconstructed images
        loss = criterion(outputs, images)     # Calculate reconstruction loss

        optimizer.zero_grad()   # Clear previous gradients
        loss.backward()         # Backpropagate to compute gradients
        optimizer.step()        # Update model weights

        epoch_loss += loss.item() * images.size(0)  # Accumulate total loss weighted by batch size

    # Print average loss per image for the epoch
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {epoch_loss / len(train_loader.dataset):.4f}")


In [ ]:
# Set model to evaluation mode (disables dropout, batchnorm updates, etc.)
model.eval()

reconstruction_errors = []  # List to store reconstruction errors for each test image

# Disable gradient calculation for faster inference and lower memory usage
with torch.no_grad():
    # Iterate through test dataset using a progress bar
    for image in tqdm(test_loader):
        image = image.to(device)           # Move image to GPU/CPU
        output = model(image)              # Get reconstructed image from the autoencoder
        loss = criterion(output, image)   # Compute reconstruction error (MSE)
        reconstruction_errors.append(loss.item())  # Save the loss value for this image


In [ ]:
# Determine anomaly detection threshold based on reconstruction errors
# Images with error below threshold are labeled '1' (normal), others '0' (anomaly)

threshold = np.mean(reconstruction_errors) + 2 * np.std(reconstruction_errors)  
# Threshold set as mean plus two standard deviations (a common heuristic)

# Generate predictions: 1 if reconstruction error is less than threshold, else 0
predictions = [1 if err < threshold else 0 for err in reconstruction_errors]

# Prepare submission file by copying test dataframe and adding predicted labels
submission_df = test_df.copy()
submission_df['label'] = predictions

# Save predictions to CSV file for submission
submission_df.to_csv('submission.csv', index=False)

# Display first few rows of the submission file
print(submission_df.head())


In [ ]:
# Prepare submission DataFrame with correct column name and order

submission_df = test_df.copy()          # Copy original test dataframe
submission_df['# label'] = predictions  # Add predictions under the column named '# label'

# Reorder columns to have 'image_id' first, then '# label'
submission_df = submission_df[['image_id', '# label']]

# Save submission file without the index column
submission_df.to_csv('submission.csv', index=False)


In [ ]:
# Save submission
submission_df.to_csv('/kaggle/working/submission.csv', index=False)

# Save model
torch.save(model.state_dict(), '/kaggle/working/best_model.pth')


In [ ]:
import os

print("Files in working directory:")
print(os.listdir('/kaggle/working'))
